In [ ]:
import purly

In [ ]:
purly.state.Machine().daemon()

In [ ]:
purly.display.output('ws://127.0.0.1:8000/model/stream')

In [ ]:
layout = purly.Layout('ws://127.0.0.1:8000/model/stream')

In [ ]:
div = layout.html('div')
layout.children.append(div)
div.style.update(height='20px', width='20px', background_color='coral')

In [ ]:
layout.sync()